In [18]:
from snowflake.snowpark import Session, functions as F
from snowflake.snowpark.functions import col
from copy import copy

In [ ]:
"""
# Establish session
connection_parameters = {
    'account': 'your_account',
    'user': 'your_user',
    'password': 'your_password',
    'role': 'your_role',
    'warehouse': 'your_warehouse',
    'database': 'demo_db',
    'schema': 'demo_schema'
}
session = Session.builder.configs(connection_parameters).create() */
"""

In [5]:
session = Session.builder.config("connection_name", "mytest").create()

In [10]:
# Data Manipulation
df = session.table("demo_db.demo_schema.sales")
result_df = df.group_by("region").agg(F.sum("amount").alias("total_sales"))
result_df.show()

----------------------------
|"REGION"  |"TOTAL_SALES"  |
----------------------------
|North     |2000.0         |
|South     |3000.0         |
|East      |4000.0         |
|West      |5000.0         |
----------------------------



In [14]:
result_df = df.filter(col("REGION") != "North")\
		.group_by(("Region")).count()\
        #.group_by(("Region"), "STATE").count()\
		#.write.mode("overwrite").save_as_table(outputTableName)
	  #return outputTableName + " table successfully written"

In [17]:
result_df.show()

----------------------
|"REGION"  |"COUNT"  |
----------------------
|South     |2        |
|East      |2        |
|West      |2        |
----------------------



In [19]:
# Optionally, you could write the results back to a table
result_df.write.save_as_table("demo_db.demo_schema.sales_summary", mode="overwrite")

In [36]:
# copy the DataFrame if you want to do a self-join
from copy import copy
result_df_copied = copy(result_df)
#>>> df_self_joined = result_df.join(result_df_copied, (df_lhs.col("key") == result_df_copied.col("key")) & (df_lhs.col("value1") == df_lhs_copied.col("value1")))
df_self_joined = result_df.join(result_df_copied,(result_df.col("region") == result_df_copied.col("region")))\
.select((result_df["Region"]).alias("Region"), (result_df_copied["Count"]).alias("Count"))
df_self_joined.print_schema()
df_self_joined.show()


root
 |-- "REGION": StringType(16777216) (nullable = True)
 |-- "COUNT": LongType() (nullable = False)
----------------------
|"REGION"  |"COUNT"  |
----------------------
|South     |2        |
|East      |2        |
|West      |2        |
----------------------



In [45]:
result_df.filter((col("Region") == "North") & (col("Amount") > 1000))
result_df.show()

----------------------
|"REGION"  |"COUNT"  |
----------------------
|South     |2        |
|East      |2        |
|West      |2        |
----------------------



In [47]:
#session.table("sample_product_data").select(upper(col("name")).alias("upper_name")).collect()
[Row(UPPER_NAME='PRODUCT 1'), Row(UPPER_NAME='PRODUCT 1A'), Row(UPPER_NAME='PRODUCT 1B'), Row(UPPER_NAME='PRODUCT 2'), Row(UPPER_NAME='PRODUCT 2A'), Row(UPPER_NAME='PRODUCT 2B'), Row(UPPER_NAME='PRODUCT 3'), Row(UPPER_NAME='PRODUCT 3A'), Row(UPPER_NAME='PRODUCT 3B'), Row(UPPER_NAME='PRODUCT 4'), Row(UPPER_NAME='PRODUCT 4A'), Row(UPPER_NAME='PRODUCT 4B')]

NameError: name 'upper' is not defined